In [ ]:
!pip install mlflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.2/28.2 MB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 54.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 677.6/677.6 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.6 MB/s eta 0:00:00


In [ ]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
import optuna
import mlflow

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device


device(type='cpu')

In [ ]:
df = pd.read_csv(r"/content/sample_data/fashion-mnist_test.csv")
df=df.fillna(0)
df.shape

(10000, 785)

In [ ]:
x=df.drop(['label'],axis=1)
y=df['label']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
X_train = torch.tensor(X_train.values, dtype=torch.float32)
X_test = torch.tensor(X_test.values, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.long)
y_test = torch.tensor(y_test.values, dtype=torch.long)

In [ ]:
class Model(nn.Module):
  def __init__(self, input_dim, hidden_dim):
    super(Model, self).__init__()
    self.net = nn.Sequential(
        nn.Linear(input_dim, hidden_dim),
        nn.ReLU(),
        nn.BatchNorm1d(hidden_dim),
        nn.Linear(hidden_dim, 10)
    )
  def forward(self, x):
    return self.net(x)

In [ ]:
class customdataset(Dataset):
  def __init__(self, x, y):
    self.x = x
    self.y = y
  def __len__(self):
    return len(self.x)
  def __getitem__(self, idx):
    return self.x[idx], self.y[idx]

In [ ]:
def objective(trial):
  learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
  hidden_nodes = trial.suggest_int("hidden_node", 1, 65,step=5)
  weight_decay=trial.suggest_float("weight_decay", 1e-5, 1e-1, log=True)
  batch_size=trial.suggest_int("batch_size", 16, 256,step=16)
  epochs=trial.suggest_int("epochs", 10, 100,step=10)

  model = Model(784, hidden_nodes).to(device)
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
  train_loader = customdataset(X_train, y_train)
  test_loader = customdataset(X_test, y_test)
  train_loader = DataLoader(train_loader, batch_size=batch_size, shuffle=True)
  test_loader = DataLoader(test_loader, batch_size=batch_size, shuffle=True)

  for epoch in range(epochs):
    for i, (x, y) in enumerate(train_loader):
      x = x.to(device)
      y = y.to(device)
      optimizer.zero_grad()
      output = model(x)
      loss = criterion(output, y)
      loss.backward()
      optimizer.step()
  model.eval()
  with torch.no_grad():
    correct = 0
    total = 0
    for x, y in test_loader:
      x = x.to(device)
      y = y.to(device)
      output = model(x)
      correct+= torch.sum(torch.argmax(output, dim=1) == y).item()
      total+= y.size(0)
    accuracy = correct / total
  return accuracy

In [ ]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=5)

[I 2025-03-27 18:14:11,966] A new study created in memory with name: no-name-2fc24b90-6b12-41a6-bcc9-a31c0659ee7f
[I 2025-03-27 18:14:16,278] Trial 0 finished with value: 0.8435 and parameters: {'learning_rate': 0.024898843504347828, 'hidden_node': 26, 'weight_decay': 2.3157220926737257e-05, 'batch_size': 96, 'epochs': 10}. Best is trial 0 with value: 0.8435.
[I 2025-03-27 18:14:35,217] Trial 1 finished with value: 0.8425 and parameters: {'learning_rate': 0.00020410412140628564, 'hidden_node': 46, 'weight_decay': 4.47565044302714e-05, 'batch_size': 32, 'epochs': 30}. Best is trial 0 with value: 0.8435.
[I 2025-03-27 18:14:42,015] Trial 2 finished with value: 0.7995 and parameters: {'learning_rate': 0.0015143683455443174, 'hidden_node': 26, 'weight_decay': 0.08243902105145057, 'batch_size': 32, 'epochs': 10}. Best is trial 0 with value: 0.8435.
[I 2025-03-27 18:14:46,713] Trial 3 finished with value: 0.781 and parameters: {'learning_rate': 0.015159240954731408, 'hidden_node': 31, 'weigh

In [ ]:
study.best_params

{'learning_rate': 0.024898843504347828,
 'hidden_node': 26,
 'weight_decay': 2.3157220926737257e-05,
 'batch_size': 96,
 'epochs': 10}

In [ ]:
study.best_value

0.8435

## TRYING OPTUNA WITH MLFLOW

In [ ]:
mlflow.set_experiment("ANN_Hyperparameter_Search")

2025/03/27 18:37:46 INFO mlflow.tracking.fluent: Experiment with name 'ANN_Hyperparameter_Search' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///content/mlruns/869391938091384374', creation_time=1743100666116, experiment_id='869391938091384374', last_update_time=1743100666116, lifecycle_stage='active', name='ANN_Hyperparameter_Search', tags={}>

In [ ]:
def mlflow_objective(trial):
  learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
  hidden_nodes = trial.suggest_int("hidden_node", 1, 65,step=5)
  weight_decay=trial.suggest_float("weight_decay", 1e-5, 1e-1, log=True)
  batch_size=trial.suggest_int("batch_size", 16, 256,step=16)
  epochs=trial.suggest_int("epochs", 10, 100,step=10)
  with mlflow.start_run(run_name=f"trial_{trial.number}"):
      mlflow.log_param("learning_rate", learning_rate)
      mlflow.log_param("hidden_node", hidden_nodes)
      mlflow.log_param("weight_decay", weight_decay)
      mlflow.log_param("batch_size", batch_size)
      mlflow.log_param("epochs", epochs)
      model = Model(784, hidden_nodes).to(device)
      criterion = nn.CrossEntropyLoss()
      optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
      train_loader = customdataset(X_train, y_train)
      test_loader = customdataset(X_test, y_test)
      train_loader = DataLoader(train_loader, batch_size=batch_size, shuffle=True)
      test_loader = DataLoader(test_loader, batch_size=batch_size, shuffle=True)

      for epoch in range(epochs):
        for i, (x, y) in enumerate(train_loader):
          x = x.to(device)
          y = y.to(device)
          optimizer.zero_grad()
          output = model(x)
          loss = criterion(output, y)
          loss.backward()
          optimizer.step()
      model.eval()
      with torch.no_grad():
        correct = 0
        total = 0
        for x, y in test_loader:
          x = x.to(device)
          y = y.to(device)
          output = model(x)
          correct+= torch.sum(torch.argmax(output, dim=1) == y).item()
          total+= y.size(0)
        accuracy = correct / total
      return accuracy

In [ ]:
study2 = optuna.create_study(direction="maximize")
study2.optimize(mlflow_objective, n_trials=10)

[I 2025-03-27 18:39:55,465] A new study created in memory with name: no-name-a101c35f-15c1-4f0b-ba1f-037a8feb7ba6
/usr/local/lib/python3.11/dist-packages/optuna/distributions.py:699: UserWarning: The distribution is specified by [1, 65] and step=5, but the range is not divisible by `step`. It will be replaced by [1, 61].
  warnings.warn(
[I 2025-03-27 18:40:20,680] Trial 0 finished with value: 0.791 and parameters: {'learning_rate': 0.026735638028807587, 'hidden_node': 26, 'weight_decay': 1.5483479465211265e-05, 'batch_size': 240, 'epochs': 90}. Best is trial 0 with value: 0.791.
/usr/local/lib/python3.11/dist-packages/optuna/distributions.py:699: UserWarning: The distribution is specified by [1, 65] and step=5, but the range is not divisible by `step`. It will be replaced by [1, 61].
  warnings.warn(
[I 2025-03-27 18:40:31,010] Trial 1 finished with value: 0.75 and parameters: {'learning_rate': 0.00011832241410384031, 'hidden_node': 6, 'weight_decay': 0.027903332885186108, 'batch_size

KeyboardInterrupt: 

In [ ]:
from mlflow.tracking import MlflowClient

client = MlflowClient()
experiment = client.get_experiment_by_name("ANN_Hyperparameter_Search")
experiment_id = experiment.experiment_id

# Fetch all runs for the experiment
runs = client.search_runs(experiment_ids=[experiment_id])

In [ ]:
for run in runs:
  print(run)

<Run: data=<RunData: metrics={}, params={'batch_size': '192',
 'epochs': '70',
 'hidden_node': '31',
 'learning_rate': '0.00010055486361762427',
 'weight_decay': '0.0021344953086871715'}, tags={'mlflow.runName': 'trial_6',
 'mlflow.source.name': '/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py',
 'mlflow.source.type': 'LOCAL',
 'mlflow.user': 'root'}>, info=<RunInfo: artifact_uri='file:///content/mlruns/869391938091384374/d1136e0e4a3e47a093dbced303e86efa/artifacts', end_time=1743101031279, experiment_id='869391938091384374', lifecycle_stage='active', run_id='d1136e0e4a3e47a093dbced303e86efa', run_name='trial_6', run_uuid='d1136e0e4a3e47a093dbced303e86efa', start_time=1743101023754, status='FAILED', user_id='root'>, inputs=<RunInputs: dataset_inputs=[]>>
<Run: data=<RunData: metrics={}, params={'batch_size': '208',
 'epochs': '100',
 'hidden_node': '46',
 'learning_rate': '0.014276194751939144',
 'weight_decay': '0.029239744599929673'}, tags={'mlflow.runName': 'trial_5'